In [1]:
from config.settings import SRC_DIR
import os
os.chdir(SRC_DIR)

In [2]:
from document_processing_system.components.data_ingestion import DataIngestion
from document_processing_system.components.data_extraction import DataExtraction

pdf = 'PO166939-204865'

data_ingester = DataIngestion(f'qc_data/qc_templates/{pdf}.pdf')
data_extracter = DataExtraction(data_ingester)
pages, tables, images = data_extracter.extract_data()
all_items = data_extracter.filter_pages(pages)
data_extracter.display_data(all_items)

Extracting pages:   0%|          | 0/15 [00:00<?, ?pages/s]

Filtering pages:   0%|          | 0/15 [00:00<?, ?pages/s]


--------Page: 2-----------
Intake Pallet QC Inspection Report
Customer DPS PO 166939 / ISS PO 204865
Supplier Code : GUIM€
Supplier : Guimera Fruits
COO : Spain
Vehicle No : Received : 03/09/2024 09:59:25
Vessel : Inspection Date : 04/09/2024 06:35:59 Print date : 04/09/2024
Apricots 20x320g Punnet
ISS Pallet ID : 7761770 Freshness Technology :
Supplier Pallet ID : 5808239 Punnet / Pad Type : N/A /
Customer Pallet ID : 5808239 Outer :
Variety : Fardao Brand : CORE
Grower : / Organic? : NO Does Pallet Meet Spec? : YES
GGN : / PLU? :
BLUE Expected Qty 80
Orchard/Farm : End Customer : Tesco
Received Qty 80
Harvest Date : 30/08/2024 DP : 7
Total Defects : 5.00%
Size/Calibre : 40/45 Packhouse :
Estimated Yield : 100%
Lot Number : 508164 Inspector : Hanna.Dziuba
Minor : Dry Splits: 1.67%
Minor : Puncture: 1.67%
Major : Scarring: 1.67%
Defects Tot : 0.00% Defects Fruit Total : 0 Packs With Defects : 0.00%
Waste Tot : 0.00% Waste Fruit Total : 0 Packs With Waste : 0%
Minor Defects Tot : 3.33%

In [3]:
documnet = ''

for page in all_items[:2]:
    documnet += page

print(documnet)


--------Page: 2-----------
Intake Pallet QC Inspection Report
Customer DPS PO 166939 / ISS PO 204865
Supplier Code : GUIM€
Supplier : Guimera Fruits
COO : Spain
Vehicle No : Received : 03/09/2024 09:59:25
Vessel : Inspection Date : 04/09/2024 06:35:59 Print date : 04/09/2024
Apricots 20x320g Punnet
ISS Pallet ID : 7761770 Freshness Technology :
Supplier Pallet ID : 5808239 Punnet / Pad Type : N/A /
Customer Pallet ID : 5808239 Outer :
Variety : Fardao Brand : CORE
Grower : / Organic? : NO Does Pallet Meet Spec? : YES
GGN : / PLU? :
BLUE Expected Qty 80
Orchard/Farm : End Customer : Tesco
Received Qty 80
Harvest Date : 30/08/2024 DP : 7
Total Defects : 5.00%
Size/Calibre : 40/45 Packhouse :
Estimated Yield : 100%
Lot Number : 508164 Inspector : Hanna.Dziuba
Minor : Dry Splits: 1.67%
Minor : Puncture: 1.67%
Major : Scarring: 1.67%
Defects Tot : 0.00% Defects Fruit Total : 0 Packs With Defects : 0.00%
Waste Tot : 0.00% Waste Fruit Total : 0 Packs With Waste : 0%
Minor Defects Tot : 3.33%

In [4]:
import os
from document_processing_system.components.model_inference import ModelInference
from config.settings import DATA_DIR

# Define the PDF folder path
pdf_folder = os.path.join(DATA_DIR, 'qc_data', 'qc_reports_output', pdf, 'qc_reports_14b')

# Ensure the directory exists
os.makedirs(pdf_folder, exist_ok=True)

# Initialize the model
# model = ModelInference(model_name="Qwen/Qwen2.5-14B")

In [ ]:
from config.purpose.multi_page_dps_qc_report_config import get_qc_prompt
from utils.post_processing import extract_json_objects

class FullDocumentModelInference(ModelInference):
    def __init__(self, model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B", max_tokens=10_000):
        super().__init__(model_name)
        self.max_tokens = max_tokens

    def generate_json(self, text):
        """Overrides `generate_json` to allow max_tokens modification"""
        outputs = self.llm_pipeline(
            text,
            max_new_tokens=self.max_tokens
        )
        response = outputs[0]["generated_text"]
        # json_response = extract_json_objects(response)
        return response

    def process_items(self, document):
        """Processes a full document and extracts structured data."""
        prompt = get_qc_prompt(document)
        response = self.generate_json(prompt)
        return response


In [6]:
# Process items
model = FullDocumentModelInference(model_name="Qwen/Qwen2.5-14B")
response = model.process_items(documnet)

INFO:root:Loading model and tokenizer...
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0
INFO:root:Model loaded successfully!


In [7]:
print(response)

I have multiple pages get the list of itemsYou are an expert quality control data extraction specialist. Your task is to extract structured quality control details from the provided text, which represents multiple pages from a QC report. Each page may contain details for multiple products. The report originates from a fresh produce distribution center and is used for internal quality control purposes.

You must return a **valid JSON array** where each product is a separate JSON object. **Do not return extra text, explanations, or comments.**

### **Schema:**
[
  {
    "product_name": "string", // The name of the product (e.g., 'Strawberries', 'Organic Blueberries') along with quantity (eg: 20x227g Punnet, 20x320g Punnet).
    "RAG": "string", // A color code indicating quality status (e.g., 'RED', 'GREEN', 'BLUE', 'AMBER'). If not explicitly a color, use null.
    "expected_qty": "string", // The expected quantity of the product.
    "received_qty": "string", // The quantity of the pro

In [ ]:
import pandas as pd
flat_data = [item for sublist in response for item in sublist]

data = pd.DataFrame(flat_data)
data

,product_name,RAG,expected_qty,received_qty,supplier_code,supplier,coo,received_date,inspection_date,print_date,...,minor_fruit_total,major_defects_tot,major_fruit_total,packs_with_major,box_pack_weights,weight_readings,fruit_weights,qa_comments,packs_fruits_inspected_sample_size,boxes_inspected
0,Strawberries 20x227g Punnet,GREEN,1000,1000,12345,ABC Farms,USA,2023-04-15 10:30:00,2023-04-15 11:00:00,2023-04-15,...,15,20,10,5,"227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","Good quality, no major defects.",100,10
1,Strawberries 20x227g Punnet,GREEN,1000,1000,12345,ABC Farms,USA,2023-04-15 10:30:00,2023-04-15 11:00:00,2023-04-15,...,15,20,10,5,"227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","Good quality, no major defects.",100,10
2,Strawberries 20x227g Punnet,GREEN,1000,1000,12345,ABC Farms,USA,2023-04-15 10:30:00,2023-04-15 11:00:00,2023-04-15,...,15,20,10,5,"227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","Good quality, no major defects.",100,10
3,Strawberries 20x227g Punnet,GREEN,1000,1000,12345,ABC Farms,USA,2023-04-15 10:30:00,2023-04-15 11:00:00,2023-04-15,...,15,20,10,5,"227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","Good quality, no major defects.",100,10
4,Strawberries 20x227g Punnet,GREEN,1000,1000,12345,ABC Farms,USA,2023-04-15 10:30:00,2023-04-15 11:00:00,2023-04-15,...,15,20,10,5,"227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","Good quality, no major defects.",100,10
5,Strawberries 20x227g Punnet,GREEN,1000,1000,12345,ABC Farms,USA,2023-04-15 10:30:00,2023-04-15 11:00:00,2023-04-15,...,15,20,10,5,"227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","227g, 227g, 227g, 227g, 227g","Good quality, no major defects.",100,10
